In [1]:
import numpy as np
import cv2
import glob
from tensorflow.keras.datasets import mnist
import pandas as pd

In [2]:
#step1 : check the dataset shape -> ok
data = glob.glob('./dataset/crop/*.jpg')
for i in data:
    img = cv2.imread(i,0)# read the data as 1 channel
    #print(img.shape)

In [3]:
#step2 : read the csv file
df = pd.read_csv('./dataset/label_crop.csv')
df.columns = ["Y","X"]

In [4]:
# 載入 MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#print(x_train.shape,y_train.shape)
#print(x_train[0],y_train[0])

In [5]:
# add new data to x_train , y_train
x_train = x_train.tolist()
y_train = y_train.tolist()
for idx in range(len(df)):
    img_path = f'./dataset/crop/{df.iloc[idx]["Y"]}'
    label = df.iloc[idx]["X"]
    img = cv2.imread(img_path,0)
    x_train.append(img)
    y_train.append(label)
x_train = np.array(x_train,dtype=np.uint8)
y_train = np.array(y_train,dtype=np.uint8)
#print(x_train.shape)
#print(y_train.shape)
#print(type(y_train[-1]))
#print(x_train[-1].dtype)

In [6]:
# check the last data -> img and label are both 3
#print(x_train[-1])
#print(y_train[-1])

In [ ]:
# training the model
TF_LITE_MODEL = './mnist.tflite'  # 要產生的 TF Lite 檔名
SAVE_KERAS_MODEL = True  # 是否儲存 Keras 原始模型
import autokeras as ak
import tensorflow as tf
# 訓練 AutoKeras 模型
clf = ak.ImageClassifier(max_trials=1, overwrite=True)
clf.fit(x_train, y_train, epochs=20)
# 用測試集評估模型
loss, accuracy = clf.evaluate(x_test, y_test)
print(f'\nPrediction loss: {loss:.3f}, accurcy: {accuracy*100:.3f}%\n')
# 匯出 Keras 模型
model = clf.export_model()
model.summary()
# 儲存 Keras 模型
if SAVE_KERAS_MODEL:
    model.save('./mnist_model')
# 將模型轉為 TF Lite 格式
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# 你也可以讀取已儲存的 Keras 模型來轉換：
# converter = tf.lite.TFLiteConverter.from_saved_model('./mnist_model')
tflite_model = converter.convert()
# 儲存 TF Lite 模型
with open(TF_LITE_MODEL, 'wb') as f:
    f.write(tflite_model)